Following Tutorial from: https://graphneural.network/getting-started/

Date: 21/04/22

In [ ]:
!pip install spektral

In [ ]:
from spektral.datasets import TUDataset
dataset = TUDataset('PROTEINS')

dataset

In [ ]:
dataset[0]

In [ ]:
dataset[:100]

In [ ]:
dataset.filter(lambda g: g.n_nodes < 500)

In [ ]:
dataset

In [ ]:
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)

In [ ]:
max_degree

In [ ]:
from spektral.transforms import Degree
# augment nodes with one-hot-encoded degree
dataset.apply(Degree(int(max_degree)))

In [ ]:
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [ ]:
class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [ ]:
model = MyFirstGNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

In [ ]:
# must user data loader
from spektral.data import BatchLoader
loader = BatchLoader(dataset[:1000], batch_size=32)

In [ ]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

In [ ]:
from spektral.data import BatchLoader

loader = BatchLoader(dataset[:1000], batch_size=32)

In [ ]:
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

In [ ]:
model.predict(BatchLoader(dataset[:1100], batch_size=32).load(), steps=loader.steps_per_epoch)